In [ ]:
pip install yfinance plotly pandas

In [34]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Helper function to map range to yfinance interval & period
def get_yf_params(range_str):
    mapping = {
        "1d": ("1m", "1d"),
        "1w": ("5m", "7d"),
        "1m": ("1h", "30d"),
        "1y": ("1d", "1y"),
        "5y": ("1wk", "5y"),
        "max": ("1mo", "max")
    }
    return mapping.get(range_str, ("1d", "1mo"))

# Main function
def plot_stock_chart(ticker_symbol, range_str="1y"):
    interval, period = get_yf_params(range_str)

    stock = yf.Ticker(ticker_symbol)
    hist = stock.history(period=period, interval=interval)

    if hist.empty:
        print("No data available.")
        return

    # Reset index to get Date column
    hist = hist.reset_index()
    # The column containing date/time information is named 'Datetime' for '1d' range
    date_col_name = "Datetime" if "Datetime" in hist.columns else "Date"
    hist[date_col_name] = pd.to_datetime(hist[date_col_name])


    # Subsample if needed to ensure ~20+ ticks
    if len(hist) > 100:
        hist = hist.iloc[::len(hist)//100]

    # Create subplots with a shared x-axis and a secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(
        x=hist[date_col_name], y=hist["Close"],
        mode='lines',
        name=ticker_symbol.upper()
    ), secondary_y=False) # Plot price on the primary y-axis

    fig.add_trace(go.Bar(
        x=hist[date_col_name], y=hist["Volume"],
        name="Volume",
        opacity=0.3
    ), secondary_y=True) # Plot volume on the secondary y-axis


    fig.update_layout(
        title=f"{ticker_symbol.upper()} Stock Price and Volume ({range_str})",
        xaxis_title="Date",
        yaxis_title="Price",
        xaxis=dict(nticks=25),
        template="plotly_white"
    )

    # Set y-axis titles
    fig.update_yaxes(title_text="Price", secondary_y=False)
    fig.update_yaxes(title_text="Volume", secondary_y=True)


    fig.show()


def levermann_score(value, ratio_name):
    try:
        if ratio_name == "P/E Ratio":
            if value is None: return 0
            return 1 if value < 12 else 0 if value <= 16 else -1
        elif ratio_name == "P/B Ratio":
            if value is None: return 0
            return 1 if value < 1 else 0 if value <= 1.5 else -1
        elif ratio_name == "Return on Equity (ROE)":
            if value is None: return 0
            return 1 if value > 0.20 else 0 if value >= 0.10 else -1
        elif ratio_name == "Operating Margin":
            if value is None: return 0
            return 1 if value > 0.12 else 0 if value >= 0.06 else -1
        elif ratio_name == "Debt-to-Equity":
            if value is None: return 0
            equity_ratio = 1 / (1 + value)
            return 1 if equity_ratio > 0.25 else 0 if equity_ratio >= 0.15 else -1
        else:
            return ""
    except:
        return ""

def get_ratios_with_levermann(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    info = ticker.info

    company_name = info.get("longName", "N/A")
    symbol = info.get("symbol", ticker_symbol)

    ratios = {
        "P/E Ratio": info.get("trailingPE"),
        "Forward P/E Ratio": info.get("forwardPE"),
        "PEG Ratio": info.get("pegRatio"),
        "P/B Ratio": info.get("priceToBook"),
        "Return on Equity (ROE)": info.get("returnOnEquity"),
        "Return on Assets (ROA)": info.get("returnOnAssets"),
        "Profit Margin": info.get("profitMargins"),
        "Operating Margin": info.get("operatingMargins"),
        "Gross Margin": info.get("grossMargins"),
        "EV/EBITDA": info.get("enterpriseToEbitda"),
        "Debt-to-Equity": info.get("debtToEquity"),
        "Current Ratio": info.get("currentRatio"),
        "Quick Ratio": info.get("quickRatio"),
        "Interest Coverage": info.get("interestCoverage"),
        "Dividend Yield": info.get("dividendYield"),
        "Payout Ratio": info.get("payoutRatio"),
        "Free Cash Flow Yield": None
    }

    df = pd.DataFrame([
        {"Ratio": k, "Value": v, "Levermann Score": levermann_score(v, k)}
        for k, v in ratios.items()
    ])

    print(f"\n📘 Company: {company_name}")
    print(f"🔢 Ticker: {symbol}")
    print(df.to_string(index=False))

# Example
ticker_input = "RHM.DE"

# Example
get_ratios_with_levermann(ticker_input)

# Example use:
plot_stock_chart(ticker_input, "1d")


📘 Company: Rheinmetall AG
🔢 Ticker: RHM.DE
                 Ratio     Value Levermann Score
             P/E Ratio 93.410440              -1
     Forward P/E Ratio 58.038100                
             PEG Ratio       NaN                
             P/B Ratio 17.335184              -1
Return on Equity (ROE)  0.220130               1
Return on Assets (ROA)  0.070620                
         Profit Margin  0.071890                
      Operating Margin  0.077660               0
          Gross Margin  0.508160                
             EV/EBITDA 43.206000                
        Debt-to-Equity 34.550000              -1
         Current Ratio  1.206000                
           Quick Ratio  0.509000                
     Interest Coverage       NaN                
        Dividend Yield  0.470000                
          Payout Ratio  0.306300                
  Free Cash Flow Yield       NaN                
